In [ ]:
# Selenium 및 웹드라이버 설치
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium datasets msal msal_extensions

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
from io import StringIO
import pandas as pd
from tqdm import tqdm
from msal import PublicClientApplication
import requests
from datasets import load_dataset
import os
from msal_extensions import FilePersistence, PersistedTokenCache

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
def get_ms_token(client_id, authority, scopes):
    # 캐시 파일 경로 설정
    cache_file_path = os.path.expanduser('~/.msal_cache.json')

    # MSAL Extensions를 사용하여 파일 기반 캐시 생성
    persistence = FilePersistence(cache_file_path)
    token_cache = PersistedTokenCache(persistence)

    # MSAL 앱 생성
    app = PublicClientApplication(client_id, authority=authority, token_cache=token_cache)

    # 캐시에서 기존 계정 확인
    accounts = app.get_accounts()
    if accounts:
        # 첫 번째 계정 선택 (여러 계정이 있을 경우 적절히 선택)
        result = app.acquire_token_silent(scopes, account=accounts[0])
        if 'access_token' in result:
            print('캐시된 토큰을 사용합니다.')
        else:
            print('캐시에서 유효한 토큰을 찾을 수 없습니다. 인증을 진행합니다...')
    else:
        # 디바이스 코드 플로우를 통한 새 인증 진행
        flow = app.initiate_device_flow(scopes=scopes)
        if 'user_code' not in flow:
            raise ValueError('디바이스 플로우 생성에 실패했습니다. 설정을 확인하세요.')
        print(f"다음 URL로 이동하여 코드를 입력하세요: {flow['verification_uri']}")
        print(f"인증 코드: {flow['user_code']}")
        result = app.acquire_token_by_device_flow(flow)

    if 'access_token' in result:
        print('인증에 성공했습니다!')
        headers = {'Authorization': f"Bearer {result['access_token']}"}
        return headers
    else:
        print('인증에 실패했습니다.')
        return None

In [ ]:
def list_onedrive_files(headers, target_name, folder_id=None):
    """
    OneDrive 폴더의 파일 목록을 가져옵니다.

    Parameters:
        headers (str): MSAL을 통해 얻은 인증 토큰
        target_name(str): 내가 찾고자 하는 파일의 이름
        folder_id (str): 폴더의 ID (None일 경우 루트 폴더)

    Returns:
        dict: 파일 이름과 파일 ID의 매핑
    """
    base_url = "https://graph.microsoft.com/v1.0/me/drive"
    url = f"{base_url}/items/{folder_id}/children" if folder_id else f"{base_url}/root/children"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        folders = response.json().get('value', [])
        # 확장자가 없는 폴더만 사전에 추가
        folder_mapping = {folder['name']: folder['id'] for folder in folders}
        print(folder_mapping)
        for name in folder_mapping:
            if name == target_name:
                return folder_mapping[name]
            elif name.find('.') == -1:
                result = list_onedrive_files(headers, target_name, folder_mapping[name])
                if result:
                    return result

    elif response.status_code == 404:
        print(f"폴더를 찾을 수 없습니다: {folder_id}")
        return False

    else:
        print(f"폴더 파일 목록을 가져오는 데 실패했습니다: {response.status_code} - {response.text}")
        return False

In [ ]:
def load_csv_from_onedrive(headers, file_id) -> pd.DataFrame:
    url = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}/content"
    response = requests.get(url, headers=headers, stream=True)

    if response.status_code == 200:
        csv = pd.read_csv(StringIO(response.text))
        print(f"파일 탐색 성공")
        return csv
    else:
        print(f"파일 다운로드 실패: {response.status_code} - {response.text}")


In [25]:
def process_korean_dataset(dataset, df):
    id_counter = len(df)    # 아이디 시작점
    result = []
    checkpoint_interval = 1000   # 체크포인트 저장 주기

     # 웹 페이지 열기
    driver = webdriver.Chrome(options=options)
    url = "https://airbnbfy.hanmesoft.com/"  # 크롤링할 사이트 URL
    driver.get(url)

    for i, data in enumerate(tqdm(dataset['document'])):
        try:
            input_text = data

            # 입력 필드 찾기 (XPATH 사용)
            input_area = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[1]/div/div[3]/textarea[1]'))
            )
            input_area.clear()
            input_area.send_keys(input_text)

            # 버튼 클릭 (XPATH 사용)
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[1]/div/div[3]/button[1]'))
            )
            # JavaScript로 클릭
            driver.execute_script("arguments[0].click();", button)

            # 결과 필드 찾기 (XPATH 사용)
            result_area = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[1]/div/div[3]/textarea[2]'))
            )
            output_text = result_area.get_attribute('value')

            result.append({'ID': f'TRAIN_{i + id_counter}', 'input': output_text, 'output': input_text})
            # 페이지 로딩 대기
            # time.sleep(2)  # 필요에 따라 조정

            if (i + 1) % checkpoint_interval == 0:
                df = save_to_csv(result, df)
                save_checkpoint(start_index + i + 1)
                result = []

        except Exception as e:
            print(f'Error at index {start_index + 1} {e}')

    # 작업 완료 후 드라이버 종료
    driver.quit()
    return None

In [ ]:
def save_to_csv(row: list[dict], df):
    DATASET_FOLDER_ID = '01UUMNEVON2CIOT46PFZHKLFI3QAGBYRUR'
    DATASET_FOLDER_URL = f"{GRAPH_API_URL}/me/drive/items/{DATASET_FOLDER_ID}"

    new_row = pd.DataFrame(row)

    # Rearrange columns to match existing DataFrame
    new_row = new_row[['ID', 'input', 'output']]

    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv('train.csv', index=False, encoding='utf-8-sig')

    # 3. OneDrive에 파일 업로드
    with open('train.csv', 'rb') as file_data:
        upload_url = f"{DATASET_FOLDER_URL}:/train.csv:/content"
        response = requests.put(upload_url, headers=headers, data=file_data)

    # 4. 업로드 결과 확인
    if response.status_code == 200 or response.status_code == 201:
        print("csv 파일 업로드 성공")
    else:
        print(f"파일 업로드 실패: {response.status_code}")
        print(f"응답 메시지: {response.text}")

    return df

In [ ]:
def load_checkpoint():
    """체크 포인트 파일에서 마지막으로 처리한 인덱스를 불러옵니다."""

    file_id = list_onedrive_files(headers, 'checkpoint.txt', '01UUMNEVON2CIOT46PFZHKLFI3QAGBYRUR')
    if not file_id:
        print("파일을 찾을 수 없습니다.")
        return 0
    else:
        url = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}/content"
        response = requests.get(url, headers=headers, stream=True)

        if response.status_code == 200:
            content = response.content.decode('utf-8')
            # 2. Extract the index (assuming it's the first line)
            index = int(content.strip())
            print(f'{index}부터 다시 시작합니다.')
            return index
        else:
            print(f"파일 다운로드 실패: {response.status_code} - {response.text}")
            return 0

In [ ]:
def save_checkpoint(index: int):
    """체크 포인트 파일에 현재 처리한 인덱스를 저장합니다."""

    DATASET_FOLDER_ID = '01UUMNEVON2CIOT46PFZHKLFI3QAGBYRUR'
    DATASET_FOLDER_URL = f"{GRAPH_API_URL}/me/drive/items/{DATASET_FOLDER_ID}"

    # 1. 체크포인트 파일에 인덱스 저장 (문자열로 변환)
    with open('checkpoint.txt', 'w') as f:
        f.write(str(index))  # 정수를 문자열로 변환

    # 2. 파일 업로드
    with open('checkpoint.txt', 'rb') as f:  # 바이너리 모드로 파일 열기
        upload_url = f"{DATASET_FOLDER_URL}:/checkpoint.txt:/content"
        response = requests.put(upload_url, headers=headers, data=f)

    # 3. 업로드 결과 확인
    if response.status_code == 200 or response.status_code == 201:
        print("체크 포인트 메모장 파일 업로드 성공")
    else:
        print(f"파일 업로드 실패: {response.status_code}")
        print(f"응답 메시지: {response.text}")

    return None

In [ ]:
# Azure 앱 정보
GRAPH_API_URL = 'https://graph.microsoft.com/v1.0'
CLIENT_ID = 'ef053b61-d7f1-4942-97d4-bb79fa475a01'  # 앱 등록에서 가져온 클라이언트 ID
AUTHORITY = 'https://login.microsoftonline.com/f09a4ef3-978d-434e-89da-a29b9f9f3c32'  # 테넌트 ID 또는 'common'
SCOPES = ['Files.ReadWrite.All']  # 필요 권한 설정

# Selenium 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
# 네이버 영화 리뷰 데이터셋 불러오기
dataset = load_dataset('nsmc')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The repository for nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
headers = get_ms_token(CLIENT_ID, AUTHORITY, SCOPES)

file_id = list_onedrive_files(headers, 'train.csv')

다음 URL로 이동하여 코드를 입력하세요: https://microsoft.com/devicelogin
인증 코드: IYV9UCP44
인증에 성공했습니다!
{'첨부 파일': '01UUMNEVPWL73GRXSYZNDZUQO7WAMFJK5U', 'dacon': '01UUMNEVP3V7HTIIDC4ND2OGQA2MDF5DEB'}
{}
{'난독화된_한글_리뷰_복원_AI': '01UUMNEVJW5NQWDSFC3RD3HRYNOWTUHNX3', '이미지_색상화_및_손실_부분_복원_AI_경진대회': '01UUMNEVILNY36PCQEUVGZB7NTLD47XF5I'}
{'.idea': '01UUMNEVLT32IT5TSGURH3LNZY2XB6NGNK', 'dataset': '01UUMNEVON2CIOT46PFZHKLFI3QAGBYRUR', 'gguf': '01UUMNEVJXS67ZKNTMMFEZZBC4675AIWY7', 'result': '01UUMNEVIFQKGRKG33TJGK3SDT37KR3INA', 'korean_review_decode_gemma_2b.ipynb': '01UUMNEVLTOW35OLYWMZAJ7MWO5AOCCOTY', 'korean_review_decode_gemma_2b.py': '01UUMNEVPP3RN2DETWUZHZURUOQIMEDLLV', 'submission_gemma_2b.csv': '01UUMNEVIN3HUGC4TKE5FJB5NCOYJ2R6WY', 'test_pd.py': '01UUMNEVPKIHMSTXETKNCZQCELVQLB773X', 'test_torch.py': '01UUMNEVNXEJDHTCR5H5FJ24CXDCYYGIW4'}
{'sample_submission.csv': '01UUMNEVLCWPWB6FU3Q5E3LWRWDYA3F6T3', 'test.csv': '01UUMNEVNMJKAHRV53NJEJ4RFYTXI4V4ID', 'train - 복사본.csv': '01UUMNEVOXV2Q6ZFM3W5G2LGRR6TZ3VBXK', 'tr

In [ ]:
# 학습 데이터를 dataframe 형태로 불러오기
train_df = load_csv_from_onedrive(headers, file_id)

파일 탐색 성공


In [26]:
start_index = load_checkpoint()
process_korean_dataset(dataset['train'][start_index:], train_df)

{'sample_submission.csv': '01UUMNEVLCWPWB6FU3Q5E3LWRWDYA3F6T3', 'test.csv': '01UUMNEVNMJKAHRV53NJEJ4RFYTXI4V4ID', 'train - 복사본.csv': '01UUMNEVOXV2Q6ZFM3W5G2LGRR6TZ3VBXK', 'train.csv': '01UUMNEVJPBF3ZVCIM2BGKRG33ADGHPYM6'}
파일을 찾을 수 없습니다.


  0%|          | 49/150000 [00:15<15:45:49,  2.64it/s]

파일 업로드 성공


  0%|          | 50/150000 [00:21<86:33:52,  2.08s/it]

파일 업로드 성공


...

파일 업로드 성공


  0%|          | 100/150000 [00:40<83:48:17,  2.01s/it]

파일 업로드 성공


  0%|          | 149/150000 [01:00<16:51:20,  2.47it/s]


KeyboardInterrupt: 

1. ID 부여하는 흐름 개선
2. 체크 포인트로 진행할 때 이전 포인트에 덮어써지는 문제 개선